## Expectation Reflection

Test ER package.

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_curve,auc
from sklearn.preprocessing import MinMaxScaler

# pypi:
#from expectation_reflection import classification as ER
#from expectation_reflection import function

# local:
from expectation_reflection_local import classification as ER
from expectation_reflection_local import function

In [2]:
np.random.seed(1)

In [3]:
Xy = np.loadtxt('kidney.dat') 
X = Xy[:,:-1]
# 2020.07.15: convert y from {-1,+1} to {0,1}:
y = (Xy[:,-1]+1)/2. 

X,y = function.make_data_balance(X,y)
print(np.unique(y,return_counts=True))

X, y = shuffle(X, y, random_state=1)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.5,random_state = 1)

sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

(array([0., 1.]), array([149, 149]))


In [4]:
model = ER.model(max_iter=10,regu=10.,random_state=10)

In [5]:
model.fit(X_train,y_train)

fit max_iter, regu, random_state: 10 10.0 10


In [6]:
y_pred = model.predict(X_test)

In [7]:
acc = accuracy_score(y_test,y_pred)

In [8]:
print(acc)

0.46308724832214765


## Hyper Parameter Optimization

In [9]:
from sklearn.model_selection import GridSearchCV

In [10]:
#model = ER.model(max_iter=100,random_state=1)
model = ER.model()

In [11]:
#regu = [0.001,0.01,0.1,0.5]
#regu = [0.001]

hyper_parameters = dict(max_iter=[5],regu=[0.001],random_state=[2,4])

In [12]:
hyper_parameters

{'max_iter': [5], 'regu': [0.001], 'random_state': [2, 4]}

In [13]:
# Create grid search using cross validation
clf = GridSearchCV(model, hyper_parameters, cv=4, iid='deprecated')

In [14]:
# Fit grid search
best_model = clf.fit(X_train, y_train)

fit max_iter, regu, random_state: 5 0.001 2
fit max_iter, regu, random_state: 5 0.001 2
fit max_iter, regu, random_state: 5 0.001 2
fit max_iter, regu, random_state: 5 0.001 2
fit max_iter, regu, random_state: 5 0.001 4
fit max_iter, regu, random_state: 5 0.001 4
fit max_iter, regu, random_state: 5 0.001 4
fit max_iter, regu, random_state: 5 0.001 4
fit max_iter, regu, random_state: 5 0.001 1


In [15]:
 # best hyper parameters
print('best_hyper_parameters:',best_model.best_params_)

best_hyper_parameters: {'max_iter': 5, 'random_state': 4, 'regu': 0.001}


In [16]:
# performance:
y_test_pred = best_model.best_estimator_.predict(X_test)
acc = accuracy_score(y_test,y_test_pred)
print(acc)

0.9395973154362416
